In [152]:
import pandas as pd
import numpy as np
import random
import re

from nltk import word_tokenize
import nltk

In [175]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [176]:
# Remove non-alpha/numeric characters from ingredients
def clean_ingredients(recipe):
    recipe = re.sub("[^0-9a-zA-Z ]+", "", recipe)
    return recipe.lower()

In [177]:
recipe_list = []
i = 0
for recipe in train.ingredients:
    ingredient_list = []
    for ingredient in recipe:
        ingredient_list.append(clean_ingredients(ingredient))
    recipe_list.append(ingredient_list)

In [178]:
train.ingredients = recipe_list

In [179]:
train.ingredients[1599]

['eggs',
 'refrigerated piecrusts',
 'salt',
 'milk',
 'sweet potatoes or yams',
 'firmly packed brown sugar',
 'i cant believe its not butter spread',
 'allpurpose flour',
 'ground cinnamon',
 'ground nutmeg',
 'vanilla extract']

In [78]:
# add all words into bag
bag_of_words = []
for recipe in train.ingredients:
    bag_of_words += recipe
#     for ingredient in recipe:
#         ingredient = word_tokenize(ingredient.lower())
#         bag_of_words+=ingredient

In [79]:
print(len(bag_of_words))
print(bag_of_words[0:20])

428275
['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles', 'plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']


In [80]:
bag_of_words = nltk.FreqDist(bag_of_words)

In [81]:
print(bag_of_words.most_common(15))

[('salt', 18049), ('onions', 7972), ('olive oil', 7972), ('water', 7457), ('garlic', 7380), ('sugar', 6434), ('garlic cloves', 6237), ('butter', 4848), ('ground black pepper', 4785), ('all-purpose flour', 4632), ('pepper', 4438), ('vegetable oil', 4385), ('eggs', 3388), ('soy sauce', 3296), ('kosher salt', 3113)]


In [82]:
# Save sorted bag of words and use first 5000 as features
word_features = [i[0] for i in bag_of_words.most_common()]
word_features = word_features[0:5000]

In [89]:
# Check if word in recipe list is in word_features
def find_features(recipe):
    ingredient = set(recipe)
    features = {}
    for i in word_features:
        features[i] = (i in ingredient)
    
    return features

In [90]:
feature_set = [(find_features(train.ingredients[i]),train.cuisine[i]) for i in range(len(train))]

In [99]:
training_set = feature_set[:int(len(feature_set) * 0.8)]
testing_set = feature_set[int(len(feature_set) * 0.8):]

In [107]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [108]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 74.51917033312382


In [109]:
classifier.show_most_informative_features(15)

Most Informative Features
            garam masala = True           indian : italia =   1200.4 : 1.0
              fish sauce = True           vietna : italia =   1176.5 : 1.0
                   mirin = True           japane : mexica =   1000.9 : 1.0
                    sake = True           japane : italia =    873.9 : 1.0
                  ginger = True           chines : italia =    791.2 : 1.0
              sesame oil = True           korean : mexica =    735.6 : 1.0
          corn tortillas = True           mexica : italia =    624.4 : 1.0
             beansprouts = True           vietna : italia =    613.0 : 1.0
       andouille sausage = True           cajun_ : italia =    605.2 : 1.0
         ground turmeric = True           indian : italia =    602.4 : 1.0
              lemongrass = True             thai : italia =    602.3 : 1.0
                 peanuts = True             thai : italia =    451.3 : 1.0
            hoisin sauce = True           chines : mexica =    412.4 : 1.0